# Applied Project in Big Data on Industrial Dataset

## DATA COLLECTION TECHNIQUES
## Part IV. Load from object storage and preprocessing

### 1. Libraries and credentials

[About boto3](https://boto3.amazonaws.com/v1/documentation/api/latest/index.html) library.

In [ ]:
import os
import sys
import json
import boto3
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
pd.set_option('display.max_columns', None)

In [ ]:
def access_data(file_path):
    with open(file_path) as file:
        access_data = json.load(file)
    return access_data

creds = access_data(file_path='access_bucket.json')
print(creds.keys())

### 2. Session and client for loading

In [ ]:
session = boto3.session.Session()
s3 = session.client(
    service_name='s3',
    aws_access_key_id=creds['aws_access_key_id'],
    aws_secret_access_key=creds['aws_secret_access_key'],
    endpoint_url='https://storage.yandexcloud.net'
)

In [ ]:
VK_DATA_BUCKET = 'apid-data-vk'

In [ ]:
all_files = [key['Key'] for key in s3.list_objects(Bucket=VK_DATA_BUCKET)['Contents']]
print('files in storage:', all_files[:10]) # works only for num of files < 1000

### 3. Load data from the storage

In [ ]:
file_to_load = all_files[1]
print('file to load:', file_to_load)
get_object_response = s3.get_object(
    Bucket=VK_DATA_BUCKET, 
    Key=file_to_load
)

In [ ]:
get_object_response

In [ ]:
data = json.load(get_object_response['Body'])
type(data)

In [ ]:
data[0]

In [ ]:
df = pd.DataFrame(data)
df.head()

### 4. Time for Big Data (Spark)

In [ ]:
from pyspark import SparkContext, SparkConf
from pyspark.sql import SparkSession
from pyspark.sql import functions as F
from pyspark.sql.types import *
from pyspark.sql.functions import udf, struct, count_distinct, from_unixtime

In [ ]:
conf = SparkConf()
conf.set('spark.master', 'local[5]')
conf.set('spark.executor.memory', '4G')
conf.set('spark.driver.memory', '4G')
sc = SparkContext(conf=conf)
spark = SparkSession(sc)
spark._jsc.hadoopConfiguration().set('fs.s3a.access.key', creds['aws_access_key_id'])
spark._jsc.hadoopConfiguration().set('fs.s3a.secret.key', creds['aws_secret_access_key'])
spark._jsc.hadoopConfiguration().set('fs.s3a.impl','org.apache.hadoop.fs.s3a.S3AFileSystem')
spark._jsc.hadoopConfiguration().set('fs.s3a.multipart.size', '104857600')
spark._jsc.hadoopConfiguration().set('fs.s3a.block.size', '33554432')
spark._jsc.hadoopConfiguration().set('fs.s3a.threads.max', '256')
spark._jsc.hadoopConfiguration().set('fs.s3a.endpoint', 'http://storage.yandexcloud.net')

In [ ]:
all_files[:10]

In [ ]:
all_files[3]

In [ ]:
file_path = file_path = f's3a://{VK_DATA_BUCKET}/' + all_files[3]
sdf = spark.read.json(file_path)

In [ ]:
sdf.show()

In [ ]:
sdf = sdf.select(F.explode(sdf.items))

In [ ]:
sdf.show()

In [ ]:
def flat_df(df, prefix=None):
    flat_cols = [c[0] for c in df.dtypes if c[1][:6] != 'struct']
    nested_cols = [c[0] for c in df.dtypes if c[1][:6] == 'struct']
    flat_df = df.select(
        flat_cols + 
        [F.col(ncol + '.' + col).alias(prefix + col if prefix else ncol + '_' + col ) 
         for ncol in nested_cols 
         for col in df.select(ncol + '.*').columns]
    )
    return flat_df

In [ ]:
sdf = flat_df(sdf, prefix='')

In [ ]:
sdf.show(3)

In [ ]:
sdf.limit(5).toPandas()

In [ ]:
sdf.count()

In [ ]:
# We can load ALL files with posts from ALL walls and users
file_path = file_path = f's3a://{VK_DATA_BUCKET}/gsom_ma-2022-11-14-05-00-20-220713/walls/wall_owner_id_*.json'
sdf = spark.read.json(file_path)

In [ ]:
sdf.count()

In [ ]:
sdf.limit(5).toPandas()

In [ ]:
sdf = sdf.select(F.explode(sdf.groups))
sdf = flat_df(sdf, prefix='')

In [ ]:
sdf.limit(5).toPandas()

### 5. Data processing and basic EDA - VK

In [ ]:
file_path = [
    f's3a://{VK_DATA_BUCKET}/gsom_spbu-2022-11-16-05-00-11-755938/wall_owner_id_*.json',
    f's3a://{VK_DATA_BUCKET}/gsom_ma-2022-11-14-05-00-20-220713/wall_owner_id_*.json',
    f's3a://{VK_DATA_BUCKET}/gsom_abiturient-2022-11-15-05-00-12-366823//wall_owner_id_*.json'
]
sdf = spark.read.json(file_path)

In [ ]:
sdf.show()

#### Creating dataset v1

In [ ]:
sdf1 = sdf.select(F.explode(sdf.groups), sdf.items)
sdf1 = flat_df(sdf1, prefix='')
sdf1.limit(5).toPandas()

In [ ]:
sdf1 = sdf1.select(sdf1.col_id, sdf1.col_name, F.explode(sdf.items))
sdf1 = flat_df(sdf1, prefix='')
sdf1.limit(5).toPandas()

In [ ]:
sdf1.groupBy("col_name").count().show()

In [ ]:
sdf1.count()

#### Creating dataset v2

In [ ]:
sdf_groups = sdf.select(F.explode(sdf.groups))
sdf_groups = flat_df(sdf_groups, prefix='')

In [ ]:
sdf_groups.limit(5).toPandas()

In [ ]:
sdf_posts = sdf.select(F.explode(sdf.items))
sdf_posts = flat_df(sdf_posts, prefix='')

In [ ]:
sdf_posts.limit(5).toPandas()

In [ ]:
sdf_posts = sdf_posts.withColumn('col_owner_id', F.regexp_replace('col_owner_id', '-', ''))
sdf_posts = sdf_posts.withColumn('col_from_id', F.regexp_replace('col_from_id', '-', ''))

In [ ]:
sdf_posts.limit(5).toPandas()

In [ ]:
sdf_posts = sdf_posts.join(sdf_groups, sdf_posts.col_owner_id == sdf_groups.col_id)

In [ ]:
sdf_posts.limit(5).toPandas()

In [ ]:
sdf_posts.count()

In [ ]:
sdf_posts.groupBy("col_name").count().show()

In [ ]:
sdf_posts.select('col_name').distinct().collect()

In [ ]:
sdf_posts.filter(sdf_posts.col_name == 'Высшая школа менеджмента СПбГУ').count()

In [ ]:
sdf_posts \
    .filter(sdf_posts.col_name == 'Высшая школа менеджмента СПбГУ') \
    .limit(5) \
    .toPandas()

In [ ]:
sdf = sdf_posts.select(
    sdf_posts.col_date,
    sdf_posts.col_comments,
    sdf_posts.col_likes,
    sdf_posts.col_reposts,
    sdf_posts.col_views,
    sdf_posts.col_text,
    sdf_posts.col_name,
    sdf_posts.col_screen_name
).dropDuplicates()
sdf.limit(5).toPandas()

In [ ]:
sdf.count()

In [ ]:
sdf.groupBy("col_name").count().show()

### 6. Data processing and basic EDA - AI Jobs

In [ ]:
file_path = './ai_jobs_data/*.json'
sdf = spark.read.json(file_path)

In [ ]:
sdf.show()

In [ ]:
sdf.describe()

In [ ]:
sdf.limit(5).toPandas()

In [ ]:
df = sdf.toPandas()

In [ ]:
df.describe()

In [ ]:
df.employmentType.hist()
plt.show()